In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [107]:
prices = pd.read_csv('prices.csv')
wind = pd.read_csv('wind_power_cleaned.csv')
grid_excess = pd.read_csv('grid_excess.csv')

In [108]:
prices['date'] = pd.to_datetime(prices['date'])
wind['date'] = pd.to_datetime(wind['date'])

In [109]:
data = pd.merge(prices, wind, on=['date', 'hour'], how='outer')

#select every 18th day and save that to a new dataframe
data['date'] = pd.to_datetime(data['date'])

In [110]:
#show rows with nan values
data[data.isnull().any(axis=1)]

,date,hour,price,wind power
7160,2023-10-26,9,70.47,NaN
7161,2023-10-26,10,70.50,NaN
7162,2023-10-26,11,74.21,NaN
7163,2023-10-26,12,74.17,NaN
7223,2023-10-29,0,14.05,NaN
8760,2023-03-26,2,NaN,1293.8


In [111]:
#get a list of every 18th day in 2023
selected_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='18D')

In [112]:
#select every 18th day
scenarios_price = {}
scenarios_wind = {}
i=1
for date in selected_dates:
    scenarios_price[i] = prices[prices['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    scenarios_wind[i] = wind[wind['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    i+=1

In [113]:
scenarios_wind

{1:      wind power
 0   3558.065000
 1   3635.240000
 2   3627.525000
 3   3503.089999
 4   3352.544999
 5   3392.480000
 6   3234.825000
 7   3212.399999
 8   3210.665000
 9   3251.419999
 10  3413.909999
 11  3417.480000
 12  3260.764999
 13  3484.150000
 14  3598.519999
 15  3502.899999
 16  3540.920000
 17  3620.304999
 18  3675.289998
 19  3604.714999
 20  3339.550000
 21  3300.650000
 22  3259.195000
 23  3269.225000,
 2:      wind power
 0   2095.155000
 1   1633.454999
 2   1133.426500
 3    823.357997
 4    654.751998
 5    619.955999
 6    535.402497
 7    489.724500
 8    496.822498
 9    489.527498
 10   575.942498
 11   825.153998
 12   938.251498
 13  1062.100000
 14  1039.370000
 15  1012.774500
 16  1082.165000
 17  1097.025000
 18  1115.865000
 19  1127.475000
 20  1329.825000
 21  1692.845000
 22  1957.520000
 23  2044.590000,
 3:      wind power
 0   2581.254999
 1   2532.064999
 2   2464.859999
 3   2478.750000
 4   2380.819999
 5   1801.594998
 6   1282.575000
 7 

In [114]:
#make 20 times 20 times 2 scenarios
scenarios = {}
for price in scenarios_price:
    for wind in scenarios_wind:
        for grid in grid_excess:
            df = pd.merge(scenarios_price[price], scenarios_wind[wind], left_index=True, right_index=True)
            #add column with grid excess
            df['grid_excess'] = grid_excess[grid]
            scenarios[(price, wind, grid)] = df

In [115]:
scenarios

{(1,
  1,
  'Scenario 1'):     price   wind power  grid_excess
 0    2.01  3558.065000            1
 1    1.38  3635.240000            1
 2    0.09  3627.525000            0
 3    0.08  3503.089999            1
 4    0.05  3352.544999            1
 5    0.00  3392.480000            1
 6   -0.38  3234.825000            0
 7    0.00  3212.399999            1
 8    0.00  3210.665000            0
 9    0.04  3251.419999            1
 10   0.80  3413.909999            0
 11   0.08  3417.480000            1
 12   0.04  3260.764999            0
 13   0.00  3484.150000            1
 14   0.85  3598.519999            0
 15  23.53  3502.899999            1
 16  36.54  3540.920000            0
 17  46.03  3620.304999            1
 18  55.57  3675.289998            0
 19  54.95  3604.714999            0
 20  49.23  3339.550000            1
 21  44.99  3300.650000            1
 22  45.96  3259.195000            1
 23  35.00  3269.225000            0,
 (1,
  1,
  'Scenario 2'):     price   wind powe